In [ ]:
import pandas as pd

In [ ]:
chartevents = pd.read_csv('./chartevents.csv')

In [ ]:
code_status = chartevents[chartevents['itemid']==223758]
chartevents = 0
code_status

In [ ]:
cmo = code_status[code_status['value']=='Comfort measures only']
cmo

In [ ]:
cmo_output = cmo[['stay_id','value']]
cmo_output.rename(columns={'value':'CMO'}, inplace=True)
cmo_output = cmo_output.drop_duplicates(subset=['stay_id'])
cmo_output.reset_index(drop=True, inplace=True)
cmo_output['CMO'] = 1
cmo_output

In [ ]:
cmo_output2 = cmo[['stay_id','charttime']]
cmo_output2 = cmo_output2.drop_duplicates(subset=['stay_id'])
cmo_output2.reset_index(drop=True, inplace=True)
cmo_output2

In [ ]:
ambiguous_data = pd.read_csv('./1hr Shock Research ambiguous Dataset_type27.csv')
non_ambiguous_data = pd.read_csv('./1hr Shock Research non_ambiguous Dataset_type27.csv')

In [ ]:
print(ambiguous_data.shape)
print(non_ambiguous_data.shape)

In [ ]:
ambiguous_data = pd.merge(ambiguous_data, cmo_output, on=['stay_id'], how='left')
non_ambiguous_data = pd.merge(non_ambiguous_data, cmo_output, on=['stay_id'], how='left')

#ambiguous_data.fillna(0, inplace=True)
#non_ambiguous_data.fillna(0, inplace=True)

print(ambiguous_data.shape)
print(non_ambiguous_data.shape)

In [ ]:
ambiguous_data['time'] = pd.to_datetime(ambiguous_data['time'])
non_ambiguous_data['time'] = pd.to_datetime(non_ambiguous_data['time'])
cmo_output2['charttime'] = pd.to_datetime(cmo_output2['charttime'])

In [ ]:
def cut_active_after_cmo(x):
    x.reset_index(drop=True, inplace=True)

    if x['CMO'].max() > 0:
        cmo_charttime_series = cmo_output2[cmo_output2['stay_id']==x['stay_id'][0]]['charttime'].max()
        
        #cmo charttime 이전의 데이터 찾기
        before_cmo_data = x[x['time']<=cmo_charttime_series]

        return before_cmo_data
    else :
        return x

In [ ]:
before_cmo_data_ambiguous = ambiguous_data.groupby('stay_id').apply(cut_active_after_cmo)
before_cmo_data_non_ambiguous = non_ambiguous_data.groupby('stay_id').apply(cut_active_after_cmo)

before_cmo_data_ambiguous.reset_index(drop=True, inplace=True)
before_cmo_data_non_ambiguous.reset_index(drop=True, inplace=True)

print('ambiguous_data', len(ambiguous_data))
print('before_cmo_data_ambiguous', len(before_cmo_data_ambiguous))

print('non_ambiguous_data', len(non_ambiguous_data))
print('before_cmo_data_non_ambiguous', len(before_cmo_data_non_ambiguous))

In [ ]:
a = before_cmo_data_ambiguous['stay_id'].unique().tolist()
b = before_cmo_data_non_ambiguous['stay_id'].unique().tolist()

all_ambiguous_patient = list(set(a) - set(b))

print(len(all_ambiguous_patient))

In [ ]:
before_cmo_data_ambiguous.to_csv('./1hr Shock Research ambiguous Dataset_type27_cmo.csv', index=False)
before_cmo_data_non_ambiguous.to_csv('./1hr Shock Research non_ambiguous Dataset_type27_cmo.csv', index=False)

Concat ambiguous and non ambiguous

In [ ]:
ambiguous_data = pd.read_csv('./1hr Shock Research ambiguous Dataset_type27_cmo.csv')
non_ambiguous_data = pd.read_csv('./1hr Shock Research non_ambiguous Dataset_type27_cmo.csv')

output = pd.concat([ambiguous_data, non_ambiguous_data], axis=0, ignore_index=True)
output = output.sort_values(by=['stay_id', 'time'], ascending=[True,True])
output.reset_index(drop=True, inplace=True)

def delete_no_vaso_patients(x):
    x.reset_index(drop=True, inplace=True)

    if len(x[x['Vasopressors']>0]) > 0:
        return x
    
data_delete_no_vaso_patients = output.groupby('stay_id').apply(delete_no_vaso_patients)
data_delete_no_vaso_patients.reset_index(drop=True, inplace=True)
data_delete_no_vaso_patients['stay_id'].unique().shape



In [ ]:
def Bye_bye(x): 
    x.reset_index(drop=True, inplace=True)
    column_list = ['Blood pressure systolic', 'Blood pressure diastolic', 'Blood pressure mean']
    
    has_nan = x[column_list].isna().any(axis=1).any()

    if has_nan == False:
        return x
    
    else :
        df_notna = x.dropna(subset=column_list)
        df_notna.reset_index(drop=True, inplace=True)

        df_isna = x[x[column_list].isnull().any(axis=1)]

        if len(df_notna)!=0:
            if df_notna['Actual Troponin test'][0]==0:
                df_notna['Actual Troponin test'][0] = df_isna['Actual Troponin test'].max()

            if df_notna['Actual Creatinine test'][0]==0: 
                df_notna['Actual Creatinine test'][0] = df_isna['Actual Creatinine test'].max()

            if df_notna['Actual Lactate test'][0]==0:
                df_notna['Actual Lactate test'][0] = df_isna['Actual Lactate test'].max()

            return df_notna

cleansed_data = data_delete_no_vaso_patients.groupby('stay_id').apply(Bye_bye)
print(cleansed_data.shape)
#imputated_data = 0
cleansed_data.reset_index(drop=True, inplace=True)
print(cleansed_data['stay_id'].unique().shape)

cleansed_data.to_csv('./1hr Shock Research type27 for Cont.csv', index=False)

In [ ]:
def calculate_rsi(df):
    df.reset_index(drop=True, inplace=True)

    feature_ls = ['Heart rate', 'Blood pressure systolic', 'Blood pressure diastolic',
                  'Blood pressure mean', 'Respiratory rate', 'SpO2', 'Temperature', 'Shock index']

    len_traj = len(df)
    def rsi(series, period):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)

        avg_gain = gain.rolling(window=period, min_periods=1).mean()
        avg_loss = loss.rolling(window=period, min_periods=1).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs + 0.00000001))
        return rsi

    rsi_df = df.copy()
    for column in feature_ls:
        rsi_df[f'{column}_RSI'] = rsi(df[column], len_traj)
        rsi_df[f'{column}_RSI'].fillna(0, inplace=True)
    
    return rsi_df

cleansed_data_rsi = cleansed_data.groupby('stay_id').apply(calculate_rsi)
cleansed_data_rsi.reset_index(drop=True, inplace=True)
cleansed_data_rsi

In [ ]:
def delta_feature(x):
    x.reset_index(drop=True, inplace=True)

    vital_ls = ['Heart rate', 'Blood pressure systolic', 'Blood pressure diastolic',
                  'Blood pressure mean', 'Respiratory rate', 'SpO2', 'Temperature', 'Shock index']

    for vital_feature in vital_ls:
        x[f'{vital_feature}_delta'] = None
        x[f'{vital_feature}_delta_ratio'] = None

        first_Not_none_index = None  # 변수 선언 및 초기화
        for p in range(len(x)):
            if not pd.isna(x[vital_feature][p]):
                first_Not_none_index = p  # 변수 할당
                break
            else:
                continue

        if first_Not_none_index is not None:
            x[f'{vital_feature}_delta'][first_Not_none_index] = 0
            x[f'{vital_feature}_delta_ratio'][first_Not_none_index] = 0
            for index in range(first_Not_none_index+1, len(x)):
                x[f'{vital_feature}_delta'][index] = x[f'{vital_feature}'][index] - x[f'{vital_feature}'][index-1]
                x[f'{vital_feature}_delta_ratio'][index] = (x[f'{vital_feature}'][index] - x[f'{vital_feature}'][index-1])/(x[f'{vital_feature}'][index-1]+0.000001)

    return x

cleansed_data_delta = cleansed_data_rsi.groupby('stay_id').apply(delta_feature)
cleansed_data_delta.reset_index(drop=True, inplace=True)
cleansed_data_delta.to_csv('./1hr Shock Research type27 for Cont_delta.csv', index=False)

In [ ]:
cleansed_data_delta.columns

In [ ]:
cleansed_data_delta

In [ ]:
cleansed_data_delta.isnull().sum()

In [ ]:
import pandas as pd
cleansed_data_delta = pd.read_csv('./1hr Shock Research type27 for Cont_delta.csv')
cleansed_data_delta['Binary Shock'] = 0
cleansed_data_delta['Binary Shock'][cleansed_data_delta[(cleansed_data_delta['Shock']>0)].index] = 1

In [ ]:
def get_shock_gap_df(x):
    x.reset_index(drop=True, inplace=True)
    
    shock_index_ls = []

    for shock_index in range(len(x)):
        if x['Shock'][shock_index] > 0:
            shock_index_ls.append(shock_index)

    if len(shock_index_ls) < 2:
        return x
    
    elif len(shock_index_ls) > 1 :
        df = x.loc[shock_index_ls[0]+1 : shock_index_ls[-1]-1]
        x['Binary Shock'][df.index] = 1

        # 조건을 만족하는 인덱스를 찾음
        condition = df[(df['Blood pressure mean'] >= 75) & (df['Vasopressors'] == 0) & (df['LR_NS'] == 0) & (df['Shock'] == 0)]
        
        if len(condition) > 5 :
            condition_indices = condition.index
            # 연속적으로 6시간 이상 유지하는 경우를 찾음
            continuous_blocks = []
            current_block = []

            for idx in condition_indices:
                if (len(current_block) == 0) :
                    current_block.append(idx)
                elif (len(current_block)>0) & (idx == current_block[-1] + 1):
                    current_block.append(idx)
                else:
                    continuous_blocks.append(current_block)
                    current_block = [idx]
            
            if len(current_block) > 0:
                continuous_blocks.append(current_block)

            # 6시간 이상 연속된 블록을 찾아서 처리
            for block in continuous_blocks:
                if len(block) >= 6:
                    x.loc[block, 'Binary Shock'] = 0
                    print(x['stay_id'][0])
        
            return x
        else :
            return x

In [ ]:
output_complete_gap = cleansed_data_delta.groupby('stay_id').apply(get_shock_gap_df)
output_complete_gap.reset_index(drop=True, inplace=True)
output_complete_gap

In [ ]:
print(cleansed_data_delta['Binary Shock'].value_counts()/len(cleansed_data_delta))

print(output_complete_gap['Binary Shock'].value_counts()/len(output_complete_gap))

In [ ]:
print(cleansed_data_delta.shape)
print(output_complete_gap.shape)

In [ ]:

print(output_complete_gap.stay_id.unique().shape)
Lactate_abn_stay_id_ls = output_complete_gap[output_complete_gap['Lactate']>10000].stay_id.unique().tolist()

output_complete_gap = output_complete_gap[~output_complete_gap['stay_id'].isin(Lactate_abn_stay_id_ls)]
print(output_complete_gap.stay_id.unique().shape)

In [ ]:
len(output_complete_gap[output_complete_gap['LR_NS']==0])/len(output_complete_gap)

In [ ]:
output_complete_gap.to_csv('./1hr Shock Research type27 for Cont_delta preprocessed GAP Mean.csv', index=False)

In [ ]:
output_complete_gap